In [70]:
import os, codecs
import gensim
from gensim import corpora
from collections import defaultdict
import string
from string import punctuation
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import pandas as pd
import numpy as np
from gensim.models import ldamodel
import itertools

In [14]:
pwd

u'/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50'

#### THESE SHOULD BE ALL THE RELATIVE PATHS
***Change as needed for your file structure***

In [15]:
#'uncomment' the 'source_dir' line that corresponds to the data set you will be using

#use for TEST data set (subset of 95 books)
#source_dir = '../books/clean' + '/'
#use for full data set
#source_dir = '../../clean_books' + '/'

#same for both cases
dist_string = '10p_5000_50'
outputs_dir = '/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored' + '/'\
                + dist_string + '/'
outputs_dir

'/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/'

In [16]:
cd /Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/

/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50


In [17]:
ls

10p_5000_50.dict                    10p_5000_50.model.state
10p_5000_50.model                   10p_5000_50.model.state.sstats.npy
10p_5000_50.model.expElogbeta.npy   10p_5000_50_lst.txt
10p_5000_50.model.id2word


In [18]:
dict_ext = '.dict'
corp_ext = '_lst.txt'
model_ext = '.model'
dict_file_path = outputs_dir + dist_string + dict_ext
corp_file_path = outputs_dir + dist_string + corp_ext
model_file_path = outputs_dir + dist_string + model_ext
dict_file_path, corp_file_path, model_file_path

('/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/10p_5000_50.dict',
 '/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/10p_5000_50_lst.txt',
 '/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/10p_5000_50.model')

**Load dictionary**

In [19]:
d = corpora.dictionary.Dictionary.load(dict_file_path)
print d, type(d)

Dictionary(2218001 unique tokens: [u'', u'thus--to', u'bison-tongue', u'with,[48', u'wire-jacket']...) <class 'gensim.corpora.dictionary.Dictionary'>


**Load/Inspect Streamed Corpus**

Class object for streaming corpus

In [67]:
class CorpStreamer(object):
    
    def __init__(self, dictionary, corp_lst_fp):
        self.dictionary = dictionary
        self.corp_lst_fp = corp_lst_fp
        
    def __iter__(self):
        with codecs.open(self.corp_lst_fp, 'r', encoding='utf_8') as f:
            for line in f:
                yield line.strip('/n').split(",")
                
class VecCorpStreamer(CorpStreamer):
    
    def __init__(self, dictionary, corp_lst_fp):
        self.dictionary = dictionary
        self.corp_lst_fp = corp_lst_fp
        self.corp_stream = CorpStreamer(self.dictionary, self.corp_lst_fp) 
    
    def __iter__(self):
        for line in self.corp_stream:
            yield self.dictionary.doc2bow(line)
    

In [68]:
c = CorpStreamer(d, corp_file_path)
len(next(iter(c))), next(iter(c))

(3444,
 [u'retranscription',
  u'one',
  u'first',
  u'project',
  u'gutenberg',
  u'etexts',
  u'offically',
  u'dated',
  u'november',
  u'22',
  u'1973',
  u'officially',
  u're-released',
  u'november',
  u'22',
  u'1993',
  u'30th',
  u'anniversary',
  u'assassination',
  u'project',
  u'gutenberg',
  u'etext',
  u"kennedy's",
  u'inaugural',
  u'address',
  u"jfk's",
  u'inaugural',
  u'address',
  u'january',
  u'20',
  u'1961',
  u'12:11',
  u'est',
  u'observe',
  u'today',
  u'victory',
  u'party',
  u'celebration',
  u'freedom',
  u'',
  u'',
  u'symbolizing',
  u'end',
  u'well',
  u'beginning',
  u'',
  u'signifying',
  u'renewal',
  u'well',
  u'change',
  u'sworn',
  u'almighty',
  u'god',
  u'solemn',
  u'oath',
  u'forbears',
  u'prescribed',
  u'nearly',
  u'century',
  u'three-quarters',
  u'ago',
  u'world',
  u'different',
  u'man',
  u'holds',
  u'mortal',
  u'hands',
  u'power',
  u'abolish',
  u'forms',
  u'human',
  u'poverty',
  u'forms',
  u'human',
  u'life'

In [69]:
vc = VecCorpStreamer(d, corp_file_path)
len(next(iter(vc))), next(iter(vc))

(1334,
 [(9, 4),
  (33, 1),
  (50, 13),
  (62, 1),
  (74, 1),
  (98, 1),
  (113, 16),
  (116, 1),
  (121, 2),
  (124, 4),
  (129, 1),
  (143, 1),
  (150, 3),
  (158, 3),
  (163, 2),
  (172, 1),
  (186, 4),
  (189, 4),
  (193, 5),
  (214, 3),
  (221, 2),
  (226, 3),
  (260, 2),
  (264, 2),
  (293, 3),
  (295, 24),
  (300, 2),
  (310, 5),
  (319, 19),
  (336, 3),
  (343, 1),
  (345, 1),
  (359, 1),
  (367, 1),
  (376, 1),
  (391, 4),
  (421, 1),
  (428, 3),
  (429, 1),
  (433, 3),
  (439, 4),
  (448, 3),
  (454, 2),
  (468, 5),
  (490, 6),
  (496, 5),
  (512, 1),
  (518, 3),
  (519, 1),
  (533, 1),
  (552, 4),
  (554, 4),
  (560, 2),
  (576, 1),
  (579, 8),
  (593, 1),
  (600, 1),
  (613, 199),
  (620, 1),
  (665, 3),
  (667, 2),
  (669, 11),
  (692, 1),
  (735, 2),
  (741, 3),
  (743, 3),
  (749, 1),
  (765, 3),
  (797, 1),
  (809, 1),
  (817, 4),
  (821, 1),
  (827, 9),
  (832, 1),
  (838, 3),
  (844, 4),
  (848, 2),
  (850, 2),
  (869, 1),
  (873, 1),
  (878, 1),
  (884, 1),
  (886, 5

**Load trained model**

In [21]:
lda = ldamodel.LdaModel.load(model_file_path)
print lda

LdaModel(num_terms=2218001, num_topics=50, decay=0.5, chunksize=2000)


In [83]:
num_topics = lda.num_topics
num_topics

50

**See the topics and their most significant terms**


In [37]:
for n in range(49):
    lda.print_topic(n)

In [39]:
lda.print_topics(num_topics=50)

[(0,
  u'0.161*"" + 0.005*"one" + 0.003*"1" + 0.003*"new" + 0.003*"first" + 0.003*"would" + 0.002*"work" + 0.002*"may" + 0.002*"illustration" + 0.002*"mr"'),
 (1,
  u'0.025*"de" + 0.009*"madame" + 0.008*"n" + 0.006*"king" + 0.005*"paris" + 0.004*"la" + 0.004*"one" + 0.004*"monsieur" + 0.004*"v" + 0.003*"france"'),
 (2,
  u'0.011*"said" + 0.008*"would" + 0.007*"one" + 0.006*"could" + 0.005*"upon" + 0.005*"man" + 0.004*"know" + 0.004*"time" + 0.004*"well" + 0.004*"see"'),
 (3,
  u'0.005*"one" + 0.003*"great" + 0.003*"man" + 0.003*"vanslyperken" + 0.002*"made" + 0.002*"two" + 0.002*"king" + 0.002*"wi" + 0.002*"time" + 0.002*"like"'),
 (4,
  u'0.006*"1" + 0.005*"one" + 0.004*"2" + 0.004*"great" + 0.003*"1/2" + 0.003*"may" + 0.003*"also" + 0.003*"church" + 0.003*"two" + 0.003*"time"'),
 (5,
  u'0.005*"rachel" + 0.004*"janet" + 0.004*"one" + 0.004*"carve" + 0.003*"computer" + 0.003*"would" + 0.003*"louis" + 0.003*"people" + 0.002*"like" + 0.002*"well"'),
 (6,
  u'0.006*"one" + 0.006*"would" 

**For a given document from corpus, see most relevant topics**

In [59]:
lda.get_document_topics(c)

In [55]:
next(iter(c))

[(9, 4),
 (33, 1),
 (50, 13),
 (62, 1),
 (74, 1),
 (98, 1),
 (113, 16),
 (116, 1),
 (121, 2),
 (124, 4),
 (129, 1),
 (143, 1),
 (150, 3),
 (158, 3),
 (163, 2),
 (172, 1),
 (186, 4),
 (189, 4),
 (193, 5),
 (214, 3),
 (221, 2),
 (226, 3),
 (260, 2),
 (264, 2),
 (293, 3),
 (295, 24),
 (300, 2),
 (310, 5),
 (319, 19),
 (336, 3),
 (343, 1),
 (345, 1),
 (359, 1),
 (367, 1),
 (376, 1),
 (391, 4),
 (421, 1),
 (428, 3),
 (429, 1),
 (433, 3),
 (439, 4),
 (448, 3),
 (454, 2),
 (468, 5),
 (490, 6),
 (496, 5),
 (512, 1),
 (518, 3),
 (519, 1),
 (533, 1),
 (552, 4),
 (554, 4),
 (560, 2),
 (576, 1),
 (579, 8),
 (593, 1),
 (600, 1),
 (613, 199),
 (620, 1),
 (665, 3),
 (667, 2),
 (669, 11),
 (692, 1),
 (735, 2),
 (741, 3),
 (743, 3),
 (749, 1),
 (765, 3),
 (797, 1),
 (809, 1),
 (817, 4),
 (821, 1),
 (827, 9),
 (832, 1),
 (838, 3),
 (844, 4),
 (848, 2),
 (850, 2),
 (869, 1),
 (873, 1),
 (878, 1),
 (884, 1),
 (886, 5),
 (915, 9),
 (926, 2),
 (936, 4),
 (955, 1),
 (959, 1),
 (971, 2),
 (974, 2),
 (992, 1),

In [95]:
for doc in itertools.islice(vc, 4000, 4002):
    print lda.get_document_topics(doc)

[(7, 0.097144964366509087), (18, 0.012469962959322081), (42, 0.018698121354748461), (48, 0.86881782510165362)]
[(30, 0.010090676377715836), (42, 0.98386487180184312)]


**For a given term in vocab, see what topics are most likely/relevant**

In [96]:
lda.get_term_topics(10, minimum_probability=0.0001)

[(15, 0.00019886762622507663),
 (16, 0.00011206210917399325),
 (19, 0.00029419479790259443),
 (26, 0.00012541808389645787),
 (39, 0.00010266798469718276),
 (41, 0.00013223749945301345),
 (49, 0.00010737724306217165)]

**Inverse of above**

In [89]:
lda.get_topic_terms(1)

[(7161, 0.025255947460431125),
 (15122, 0.0086689939408860339),
 (5432, 0.0077915323697854907),
 (11048, 0.0056849631330603938),
 (9642, 0.0054322980022665866),
 (16394, 0.0042664605014468117),
 (1756, 0.0042437781134540171),
 (10582, 0.0042318873047157328),
 (1097, 0.0037023966381429033),
 (10259, 0.0031507460943194085)]

**Human-readable topic terms**

In [90]:
for tup in lda.get_topic_terms(0):
    print d[tup[0]]


one
1
new
first
would
work
may
illustration
mr


In [91]:
d[0]

u'down-stairs'

In [92]:
d.doc2bow(next(iter(c)))

[(9, 4),
 (33, 1),
 (50, 13),
 (62, 1),
 (74, 1),
 (98, 1),
 (113, 16),
 (116, 1),
 (121, 2),
 (124, 4),
 (129, 1),
 (143, 1),
 (150, 3),
 (158, 3),
 (163, 2),
 (172, 1),
 (186, 4),
 (189, 4),
 (193, 5),
 (214, 3),
 (221, 2),
 (226, 3),
 (260, 2),
 (264, 2),
 (293, 3),
 (295, 24),
 (300, 2),
 (310, 5),
 (319, 19),
 (336, 3),
 (343, 1),
 (345, 1),
 (359, 1),
 (367, 1),
 (376, 1),
 (391, 4),
 (421, 1),
 (428, 3),
 (429, 1),
 (433, 3),
 (439, 4),
 (448, 3),
 (454, 2),
 (468, 5),
 (490, 6),
 (496, 5),
 (512, 1),
 (518, 3),
 (519, 1),
 (533, 1),
 (552, 4),
 (554, 4),
 (560, 2),
 (576, 1),
 (579, 8),
 (593, 1),
 (600, 1),
 (613, 199),
 (620, 1),
 (665, 3),
 (667, 2),
 (669, 11),
 (692, 1),
 (735, 2),
 (741, 3),
 (743, 3),
 (749, 1),
 (765, 3),
 (797, 1),
 (809, 1),
 (817, 4),
 (821, 1),
 (827, 9),
 (832, 1),
 (838, 3),
 (844, 4),
 (848, 2),
 (850, 2),
 (869, 1),
 (873, 1),
 (878, 1),
 (884, 1),
 (886, 5),
 (915, 9),
 (926, 2),
 (936, 4),
 (955, 1),
 (959, 1),
 (971, 2),
 (974, 2),
 (992, 1),

In [93]:
next(itertools.islice(c, 2, 3))

[u'1609',
 u'sonnets',
 u'william',
 u'shakespeare',
 u'1',
 u'fairest',
 u'creatures',
 u'desire',
 u'increase',
 u'thereby',
 u"beauty's",
 u'rose',
 u'might',
 u'never',
 u'die',
 u'riper',
 u'time',
 u'decease',
 u'tender',
 u'heir',
 u'might',
 u'bear',
 u'memory',
 u'thou',
 u'contracted',
 u'thine',
 u'bright',
 u'eyes',
 u"feed'st",
 u'thy',
 u"light's",
 u'flame',
 u'self-substantial',
 u'fuel',
 u'making',
 u'famine',
 u'abundance',
 u'lies',
 u'thy',
 u'self',
 u'thy',
 u'foe',
 u'thy',
 u'sweet',
 u'self',
 u'cruel',
 u'thou',
 u'art',
 u"world's",
 u'fresh',
 u'ornament',
 u'herald',
 u'gaudy',
 u'spring',
 u'within',
 u'thine',
 u'bud',
 u'buriest',
 u'thy',
 u'content',
 u'tender',
 u'churl',
 u"mak'st",
 u'waste',
 u'niggarding',
 u'pity',
 u'world',
 u'else',
 u'glutton',
 u'eat',
 u"world's",
 u'due',
 u'grave',
 u'thee',
 u'2',
 u'forty',
 u'winters',
 u'shall',
 u'besiege',
 u'thy',
 u'brow',
 u'dig',
 u'deep',
 u'trenches',
 u'thy',
 u"beauty's",
 u'field',
 u'thy'